In [ ]:
import pandas as pd
import numpy as np
from textwrap import wrap
from matplotlib import pyplot as plt
from src.utils.colors import flatuicolors as colors
from src.utils.styling import hide_and_move_axis, add_label
import seaborn as sns
import matplotlib.colors as mcolor
import matplotlib as mpl
import scipy
from scipy.stats import linregress

In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
import os
import datetime
from pathlib import Path

# Initialize hydra and move to the root of the repository
try:
    hydra.initialize(version_base=None, config_path="../config/")
    CONFIG = hydra.compose(config_name="main.yaml")
    print('Initializing hydra')
except:
    print('Hydra already initalized!')
else:
    # Create an output folder in the root of the repository
    os.chdir('..')
    OUTPUT_FOLDER = Path('output/{0}'.format(datetime.datetime.now()))
    Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

In [ ]:
# Load data
df = pd.read_feather(Path(CONFIG.data.processed) / CONFIG.data.filenames.merged_data)

In [ ]:
def plot_wellbeing_per_gender(ax, df, question_key='total_wellbeing', binwidth=0.2, xmin=1, xmax=5):

    # Compute average wellbeing per user
    wellbeing_per_gender = df.groupby('user_id')[['salutation', question_key]].agg({'salutation': 'max', question_key: 'mean'})
    
    for salutation in ('M', 'F'):

        # Set specifics of plot
        sign = (1. if salutation == 'M' else -1)
        color = (colors.greensea if salutation == 'M' else colors.pumpkin)
        label = ('Male' if salutation == 'M' else 'Female')

        # Plot relative frequncy of responses
        values = wellbeing_per_gender[wellbeing_per_gender.salutation == salutation][question_key]
        count, bins = np.histogram(values, bins=np.arange(xmin - .5 * binwidth, xmax + .501 * binwidth, binwidth))
        bins = .5 * (bins[1:] + bins[:-1])
        w = .4 * np.diff(bins)[0]
        count = count / len(values)
        ax.bar(bins, count, width=sign * w, align='edge', edgecolor='w', label=label, color=color)

        mean, std = values.mean(), values.std()
        print(f'Average {question_key}: {mean} +/- {std} (std)')
        
    hide_and_move_axis(ax)

    ax.legend()
    ax.set_xlabel('WHO-5 Wellbeing')
    ax.set_ylabel('Relative Frequency')


def plot_wellbeing_per_age(ax, df, color=colors.wetasphalt):
    
    # First aggregate per user
    df_age = df.groupby('user_id')[['birth_date', 'total_wellbeing']].mean()
    
    # Then aggregate per birth_date
    df_age = df_age.groupby('birth_date').agg(['mean', 'std', 'count'])
    
    df_age.columns = df_age.columns.droplevel(0)
    df_age['err'] = 1.96 * df_age['std'] / np.sqrt(df_age['count'])

    ax.fill_between(df_age.index, df_age['mean'] - df_age['err'], df_age['mean'] + df_age['err'], alpha=.3, color=color)
    ax.plot(df_age['mean'], color=color, marker='o', markersize=4)
    
    hide_and_move_axis(ax)

    ax.set_xlabel('Birth Year')
    ax.set_ylabel('Average WHO-5 Wellbeing')
    ax.set_xticks(range(1930, 2010, 10))
    
            
def plot_survey_response_per_vitals(ax, df, vital_key, question_key, color='r', bins=40, label=None, xlabel=None, err_fac=1.96, norm=False, device=None):

    if device:
        df = df[df.deviceid == device]
        
    df = df[[vital_key, question_key]].dropna()

    norm_val = df[question_key].mean()
    
    df['bins'] = pd.cut(df[vital_key], bins)
    df.bins = df['bins'].apply(lambda x: x.left + 0.5 * (x.right - x.left))

    df = df.groupby(['bins'], observed=False)[question_key].agg(['mean', 'count', 'std'])
    
    if norm:
        #norm_val = df['mean'].max()
        df['mean'] = df['mean'] / norm_val
        df['std'] = df['std'] / norm_val

    df['err'] = err_fac * df['std'] / np.sqrt(df['count'])
    df = df[df['count'] > 100]

    ax.fill_between(df.index, df['mean'] - df['err'], df['mean'] + df['err'], alpha=.3, color=color)
    ax.plot(df['mean'], color=color, marker='o', markersize=3, label=label)

    #print(f'Correlation {vital_key}/{question_key}/{label}:', np.corrcoef(df.index, df['mean'])[0, 1])
    print(f'Regression {vital_key}/{question_key}/{label}:', linregress(df.index, df['mean']).slope)
    
    hide_and_move_axis(ax)
    ax.set_xlabel(xlabel)


def ttest(df, bins, direction, vital_key, survey_key, level):

    results = []
    df = df[[vital_key, survey_key]].dropna()
    df['bins'] = pd.cut(df[vital_key], bins)
    df.bins = df['bins'].apply(lambda x: x.left + 0.5 * (x.right - x.left))

    for bin in np.sort(df.bins.unique()):

        if np.isnan(bin): continue
        values = df[df.bins == bin]
        if len(values) < 100: continue
        if scipy.stats.ttest_1samp(values[survey_key], 0, alternative=direction).pvalue < level:
            results.append(bin)
            
    return results


def get_bins(vmin, vmax, binwidth):
    return np.arange(vmin - .5 * binwidth, vmax + .5 * binwidth, binwidth)


def plot_with_ttest(ax, df, vital_key, question_key, bins, m0=0.15, delta_m=0.035):
    
    plot_survey_response_per_vitals(ax, df, vital_key, question_key, bins=bins, color=colors.midnightblue, err_fac=1.96) 
    for color, direction in ((colors.greensea, 'greater'), (colors.pumpkin, 'less')):
        for i, level in enumerate([0.001, 0.01, 0.05]):
            for sig in ttest(df=df, bins=bins, direction=direction, vital_key=vital_key, survey_key=question_key, level=level):
                print(vital_key, question_key, level, direction, sig)
                ax.text(sig, m0 + delta_m * i, '*', size=20, horizontalalignment='center', verticalalignment='center', c=color)

# User statistics

In [ ]:
print('Total number:', df.user_id.unique().shape[0])
print('Total number males:', df[df.salutation == 'M'].user_id.unique().shape[0])
print('Share males:', df[df.salutation == 'M'].user_id.unique().shape[0] /  df.user_id.unique().shape[0])
print('Total number females:', df[df.salutation == 'F'].user_id.unique().shape[0])
print('Share females:', df[df.salutation == 'F'].user_id.unique().shape[0] /  df.user_id.unique().shape[0])
print('Unique responses:', df[['user_id', 'date']].drop_duplicates().shape[0])
print('Unique responses with RHR:', df[['date', 'v65', 'total_wellbeing']].dropna().shape[0])
print('Unique responses with steps:', df[['date', 'v9', 'total_wellbeing']].dropna().shape[0])
print('Unique responses with sleep duration:', df[['date', 'v43', 'total_wellbeing']].dropna().shape[0])
print('Unique responses with sleep onset:', df[['date', 'v52', 'total_wellbeing']].dropna().shape[0])
print('Unique responses with sleep offset:', df[['date', 'v53', 'total_wellbeing']].dropna().shape[0])

In [ ]:
# 34636 - df[~df[['v65', 'v9', 'v52', 'v53', 'v43']].isna().all(axis=1)][['user_id', 'deviceid']].drop_duplicates()['deviceid'].value_counts().sum()

# Figure 1 - Demographics / Heart Rate / Steps

In [ ]:
f, axarr = plt.subplots(2, 3, figsize=(10, 6))

question_key = 'total_wellbeing'
device = None

# First row: Wellbeing per gender
plot_wellbeing_per_gender(axarr[0, 0], df, binwidth=0.5)

for gender, color, label in (('M', colors.greensea, 'Male'), ('F', colors.pumpkin, 'Female')):
    plot_survey_response_per_vitals(axarr[0, 1], df[df.salutation==gender], 'v65', question_key, bins=np.arange(30, 91, 1), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[0, 2], df[df.salutation==gender], 'v9', question_key, bins=np.arange(0, 25000, 500), color=color, label=label, device=device)

# Second row: Wellbeing per age
plot_wellbeing_per_age(axarr[1, 0], df)

for age_group, color, label in ((0, colors.greensea, 'Young Adults'), (1, colors.pumpkin, 'Middle Ager'), (2, colors.wisteria, 'Elderly')):
    plot_survey_response_per_vitals(axarr[1, 1], df[df.age_group==age_group], 'v65', question_key, bins=np.arange(30, 91, 2), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[1, 2], df[df.age_group==age_group], 'v9', question_key, bins=np.arange(0, 25000, 1000), color=color, label=label, device=device)

# Draw age level
age_level1 = CONFIG.process.users.age_level1
age_level2 = CONFIG.process.users.age_level2
axarr[1, 0].axvline(2020 - age_level1 + 2.5, c=colors.wetasphalt, ls=':')
axarr[1, 0].axvline(2020 - age_level2 + 2.5, c=colors.wetasphalt, ls=':')
axarr[1, 0].fill_between([1930, 2020 - age_level2], 2.8, 2.9, color=colors.wisteria, alpha=0.3)
axarr[1, 0].fill_between([2020 - age_level2 + 5, 2020 - age_level1], 2.8, 2.9, color=colors.pumpkin, alpha=0.3)
axarr[1, 0].fill_between([2020 - age_level1 + 5, 2000], 2.8, 2.9, color=colors.greensea, alpha=0.3)

add_label(axarr, pos=['upper right', 'upper right', 'upper left', 'lower left', 'upper right', 'upper left'], size=22)

for ax in axarr.flatten()[1:]:
    ax.set_ylim(2.5, 3.8)
    ax.set_ylabel('Average WHO-5 Wellbeing')

for ax in axarr[:, 1]:
    ax.legend(loc='lower left')
    ax.set_xlabel('Resting heart rate [bpm]')

for ax in axarr[:, 2]:
    ax.set_xlabel('Step count')

plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / 'figure1_wellbeing_heartrate_steps.pdf')

# Figure 2 - Sleep timing and Wellbeing

In [ ]:
f, axarr = plt.subplots(2, 2, figsize=(6, 6), sharey=True)
question_key = 'total_wellbeing_Z'

# Sleep Onset
bins = get_bins(-4, 3, 1/3)
plot_with_ttest(ax=axarr[0,0], vital_key ='v52', question_key=question_key, df=df, bins=bins)

# Sleep Offset
bins = get_bins(5 + 1/3, 10, 1/3)
plot_with_ttest(ax=axarr[0,1], vital_key ='v53', question_key=question_key, df=df, bins=bins)

# Midsleep
bins = get_bins(2, 7, 1/3)
plot_with_ttest(ax=axarr[1,0], vital_key ='midsleep', question_key=question_key, df=df, bins=bins)

# Sleep duration
bins = get_bins(5+1/3, 9+1*2/3, 1/3)
plot_with_ttest(ax=axarr[1,1], vital_key ='v43_hr', question_key=question_key, df=df, bins=bins)

for ax in axarr.flatten():
    ax.axhline(0, ls=':', c=colors.midnightblue)

axarr[0, 0].set_xlabel('Sleep Onset [hrs relative to midnight]')
axarr[0, 1].set_xlabel('Sleep Offset [hrs relative to midnight]')
axarr[1, 0].set_xlabel('Midsleep [hrs relative to midnight]')
axarr[1, 1].set_xlabel('Sleep duration [hrs]')

axarr[0, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[1, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[0, 1].set_ylabel(None)
axarr[1, 1].set_ylabel(None)

add_label(axarr.flatten(), pos=['lower left', 'lower left', 'lower left', 'lower left'], size=22)
plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / f'figure2_wellbeing_sleep_{question_key}.pdf')

# Figure 3 - Variance of sleep and wellbeing

In [ ]:
f, axarr = plt.subplots(2, 2, figsize=(6, 6), sharey=True)
question_key = 'total_wellbeing_Z'

# Sleep Onset
bins = get_bins(0, 3.1, 1/3)
plot_with_ttest(ax=axarr[0,0], vital_key ='v52std', question_key=question_key, df=df, bins=bins)

# Sleep Offset
bins = get_bins(0, 3.1, 1/3)
plot_with_ttest(ax=axarr[0,1], vital_key ='v53std', question_key=question_key, df=df, bins=bins)

# Midsleep
bins = get_bins(0, 2.5, 0.25)
plot_with_ttest(ax=axarr[1,0], vital_key ='midsleepstd', question_key=question_key, df=df, bins=bins)

# Sleep duration
bins = get_bins(20, 120, 10)
plot_with_ttest(ax=axarr[1,1], vital_key ='v43std', question_key=question_key, df=df, bins=bins)

for ax in axarr.flatten():
    ax.axhline(0, ls=':', c=colors.midnightblue)

axarr[0, 0].set_xlabel('Standard Deviation in\nSleep Onset [hrs]')
axarr[0, 1].set_xlabel('Standard Deviation in\nSleep Offset [hrs]')
axarr[1, 0].set_xlabel('Standard Deviation in\nMidsleep [hrs]')
axarr[1, 1].set_xlabel('Standard Deviation in\nSleep Duration [mins]')

axarr[0, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[1, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[0, 1].set_ylabel(None)
axarr[1, 1].set_ylabel(None)

add_label(axarr.flatten(), pos=['lower left', 'lower left', 'lower left', 'lower left'], size=22)
plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / f'figure3_wellbeing_sleep_std_{question_key}.pdf')

# SI Figures

## Wellbeing for RHR/activity stratified by device

In [ ]:
f, axarr = plt.subplots(2, 5, figsize=(12, 6), sharey=True, sharex='row')

question_key = 'total_wellbeing'

for device, label, color in ((2, 'Fitbit', colors.greensea), (3, 'Garmin', colors.pumpkin), (6, 'Apple', colors.wisteria)):
    plot_survey_response_per_vitals(axarr[0, 0], df[df.salutation=='F'], 'v65', question_key, bins=np.arange(30, 91, 1), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[1, 0], df[df.salutation=='F'], 'v9', question_key, bins=np.arange(0, 25000, 500), color=color, label=label, device=device)

    plot_survey_response_per_vitals(axarr[0, 1], df[df.salutation=='M'], 'v65', question_key, bins=np.arange(30, 91, 1), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[1, 1], df[df.salutation=='M'], 'v9', question_key, bins=np.arange(0, 25000, 500), color=color, label=label, device=device)

    plot_survey_response_per_vitals(axarr[0, 2], df[df.age_group==0], 'v65', question_key, bins=np.arange(30, 91, 2), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[1, 2], df[df.age_group==0], 'v9', question_key, bins=np.arange(0, 25000, 1000), color=color, label=label, device=device)

    plot_survey_response_per_vitals(axarr[0, 3], df[df.age_group==1], 'v65', question_key, bins=np.arange(30, 91, 2), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[1, 3], df[df.age_group==1], 'v9', question_key, bins=np.arange(0, 25000, 1000), color=color, label=label, device=device)
    
    plot_survey_response_per_vitals(axarr[0, 4], df[df.age_group==2], 'v65', question_key, bins=np.arange(30, 91, 2), color=color, label=label, device=device)
    plot_survey_response_per_vitals(axarr[1, 4], df[df.age_group==2], 'v9', question_key, bins=np.arange(0, 25000, 1000), color=color, label=label, device=device)


add_label(axarr, pos=['lower right'] * 10, size=22)
axarr[0, 0].legend(loc='upper right')

for ax in axarr[:, 0]:
    ax.set_ylabel('Average WHO-5 Wellbeing')

for ax in axarr[0]:
    ax.set_xlabel('Resting heart rate [bpm]')

for ax in axarr[1]:
    ax.set_xlabel('Daily Step Count')

plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / 'figure1_wellbeing_heartrate_steps_SI.pdf')

In [ ]:
f, axarr = plt.subplots(2, 2, figsize=(6, 6), sharey=True)
question_key = 'total_wellbeing_Z'

# Sleep Onset
bins = get_bins(-4, 3, 1/3)
plot_with_ttest(ax=axarr[0,0], vital_key ='v52', question_key=question_key, df=df[df.deviceid==3], bins=bins)

# Sleep Offset
bins = get_bins(5 + 1/3, 10, 1/3)
plot_with_ttest(ax=axarr[0,1], vital_key ='v53', question_key=question_key, df=df[df.deviceid==3], bins=bins)

# Midsleep
bins = get_bins(2, 7, 1/3)
plot_with_ttest(ax=axarr[1,0], vital_key ='midsleep', question_key=question_key, df=df[df.deviceid==3], bins=bins)

# Sleep duration
bins = get_bins(5+1/3, 9+1*2/3, 1/3)
plot_with_ttest(ax=axarr[1,1], vital_key ='v43_hr', question_key=question_key, df=df[df.deviceid==3], bins=bins)

for ax in axarr.flatten():
    ax.axhline(0, ls=':', c=colors.midnightblue)

axarr[0, 0].set_xlabel('Sleep Onset [hrs relative to midnight]')
axarr[0, 1].set_xlabel('Sleep Offset [hrs relative to midnight]')
axarr[1, 0].set_xlabel('Midsleep [hrs relative to midnight]')
axarr[1, 1].set_xlabel('Sleep duration [hrs]')

axarr[0, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[1, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[0, 1].set_ylabel(None)
axarr[1, 1].set_ylabel(None)

add_label(axarr.flatten(), pos=['lower left', 'lower left', 'lower left', 'lower left'], size=22)
plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / f'figure2_wellbeing_sleep_{question_key}_SI.pdf')

# SI Figures

## Weekend - weekday difference steps

In [ ]:
f, ax = plt.subplots(1, 1, sharey=True, figsize=(5, 4))

bins = get_bins(-8000, 8001, 2000)
plot_with_ttest(ax=ax, vital_key ='v9difference', question_key='total_wellbeing_Z', df=df, bins=bins, m0=0.33)

ax.set_ylim(-.2, .45)    
ax.axhline(0, ls=':', color=colors.midnightblue)    
ax.set_ylabel('Average WHO-5 Z-Scores')
ax.set_xlabel('Activity difference weekend/weekday [steps]')

plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / 'si_figure1_wellbeing_weekend_weekday_steps.pdf')

# Sandbox

## Figure 2c: 2a and 2b in one Figure

In [ ]:
def get_bins(vmin, vmax, binwidth):
    return np.arange(vmin - .5 * binwidth, vmax + .5 * binwidth, binwidth)

f, axarr = plt.subplots(3, 4, figsize=(12, 9), sharey='row')
df['v43_hr'] = df.v43 / 60

question_key = 'total_wellbeing_Z'

# Sleep Onset
bins = get_bins(-4 + 2/3, 2 + 1/3, 1/3)
plot_survey_response_per_vitals(axarr[0, 0], df, 'v52', question_key, bins=bins, color=colors.midnightblue, err_fac=1.96)

# Sleep Offset
bins = get_bins(4 + 2/3, 9, 1/3)
plot_survey_response_per_vitals(axarr[0, 1], df, 'v53', question_key, bins=bins, color=colors.midnightblue, err_fac=1.96)

# Midsleep
bins = get_bins(0.5 + 2 * 1/3, 6 - 2/3, 1/3)
plot_survey_response_per_vitals(axarr[0, 2], df, 'midsleep', question_key, bins=bins, color=colors.midnightblue, err_fac=1.96)


# Sleep duration
bins = get_bins(5.5, 10, 0.25)
plot_survey_response_per_vitals(axarr[0, 3], df, 'v43_hr', question_key, bins=bins, color=colors.midnightblue, err_fac=1.96)


# Standard deviation over all days

vital_key = 'v52'
bins = np.arange(0, 3.1, 1/6)
plot_survey_response_per_vitals(axarr[1, 0], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

vital_key = 'v53'
bins = np.arange(0, 2.5, 1/6)
plot_survey_response_per_vitals(axarr[1, 1], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

vital_key = 'midsleep'
bins = np.arange(0, 2.1, 1/6)
plot_survey_response_per_vitals(axarr[1, 2], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

vital_key = 'v43'
bins = np.arange(0, 120, 5)
plot_survey_response_per_vitals(axarr[1, 3], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

# Standard deviation discriminated
vital_key = 'v52'
bins = np.arange(0, 3.1, 1/3)
plot_survey_response_per_vitals(axarr[2, 0], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[2, 0], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')

vital_key = 'v53'
bins = np.arange(0, 3.1, 1/3)
plot_survey_response_per_vitals(axarr[2, 1], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[2, 1], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')

vital_key = 'midsleep'
bins = np.arange(0, 2.5, 1/3)
plot_survey_response_per_vitals(axarr[2, 2], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[2, 2], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')

vital_key = 'v43'
bins = np.arange(10, 120, 10)
plot_survey_response_per_vitals(axarr[2, 3], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[2, 3], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')


# Finalize plot
axarr[0, 0].axvline(0, c=colors.midnightblue, ls=':')
axarr[0, 1].axvline(7, c=colors.midnightblue, ls=':')
axarr[0, 2].axvline(3.5, c=colors.midnightblue, ls=':')
axarr[0, 3].axvline(7.5, c=colors.midnightblue, ls=':')

axarr[0, 0].set_xlabel('Sleep Onset [hrs relative to midnight]')
axarr[0, 1].set_xlabel('Sleep Offset [hrs relative to midnight]')
axarr[0, 2].set_xlabel('Midsleep [hrs relative to midnight]')
axarr[0, 3].set_xlabel('Sleep duration [hrs]')

axarr[0, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[1, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[2, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')

add_label(axarr.flatten(), pos=12 * ['lower left'], size=22)

axarr[1, 0].set_xlabel('Standard Deviation in\nSleep Onset [hrs]')
axarr[1, 1].set_xlabel('Standard Deviation in\nSleep Offset [hrs]')
axarr[1, 2].set_xlabel('Standard Deviation in\nMidsleep [hrs]')
axarr[1, 3].set_xlabel('Standard Deviation in\nSleep Duration [mins]')
axarr[2, 0].set_xlabel('Standard Deviation in\nSleep Onset [hrs]')
axarr[2, 1].set_xlabel('Standard Deviation in\nSleep Offset [hrs]')
axarr[2, 2].set_xlabel('Standard Deviation in\nMidsleep [hrs]')
axarr[2, 3].set_xlabel('Standard Deviation in\nSleep Duration [mins]')

axarr[2, 0].legend(loc='upper right')
plt.tight_layout()

plt.savefig(OUTPUT_FOLDER / 'figure2_wellbeing_sleep_with_std.pdf')

## Figure X: 2D histograms

In [ ]:
def plot2d_histograms(ax, df, x_key, y_key, vmin=-0.3, vmax=0.1, bins=20, hline=None, vline=None):
    
    x = df[y_key]
    y = df[x_key] 
    wellbeing = df.total_wellbeing_Z

    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(wellbeing)

    x = x[mask]
    y = y[mask]
    wellbeing = wellbeing[mask]

    if isinstance(bins, list):
        bins = [bins[1], bins[0]]
    H0, xedges, yedges = np.histogram2d(x, y, weights=wellbeing, bins=bins)
    H1, xedges, yedges = np.histogram2d(x, y, bins=bins)

    H = H0 / H1
    H[H1 < 50] = np.nan
    
    xmask = np.isfinite(H).any(axis=1)
    xedges = .5 * (xedges[1:] + xedges[:-1])
    xedges = xedges[xmask]
    H = H[xmask, :]
    
    ymask = np.isfinite(H).any(axis=0)
    yedges = .5 * (yedges[1:] + yedges[:-1])
    yedges = yedges[ymask]
    H = H[:, ymask]

    norm = mcolor.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    
    dx = np.diff(xedges)[0] / 2
    dy = np.diff(yedges)[0] / 2

    extent = [yedges[0] - dy, yedges[-1] + dy, xedges[0] - dx, xedges[-1] + dx]
    cmap = mpl.colormaps['RdBu'].resampled(9)
    cmap.set_bad('0.9')
    cticks = np.round(norm.inverse(np.linspace(0, 1, 10)) , 3)[::3]
    
    sc = ax.imshow(H, origin='lower', cmap=cmap, norm=norm, extent=extent, aspect='auto', alpha=0.75)
    
    #ax.set_xticks(yedges[::4])
    if hline is not None:
        ax.axhline(hline, color=colors.midnightblue, lw=3, ls='--')
    if vline is not None:
        ax.axvline(vline, color=colors.midnightblue, lw=3, ls='--')

    plt.colorbar(sc, ax=ax, ticks=cticks, extend='both')
    

f, axarr = plt.subplots(2, 3, figsize=(15 / 1.2, 8.5 / 1.2))

ax = axarr[0, 0]
bins=[get_bins(40, 90, 2), get_bins(0, 18000, 1000)]
plot2d_histograms(ax, df, 'v65', 'v9', bins=bins, vmin=-0.4, vmax=0.4)
ax.set_xlabel('Resting heart rate [bpm]')
ax.set_ylabel('Step count')

ax = axarr[0, 1]
bins = get_bins(40, 100, 2)
plot2d_histograms(ax, df, 'v65weekday', 'v65weekend', bins=bins, vmin=-0.3, vmax=0.3)
ax.plot([46, 78], [46, 78], lw=3, color=colors.midnightblue)
ax.set_xlabel('RHR on weekdays [bpm]')
ax.set_ylabel('RHR on weekends [bpm]')

ax = axarr[0, 2]
bins = get_bins(0, 25000, 1000)
plot2d_histograms(ax, df, 'v9weekday', 'v9weekend', bins=bins, vmin=-0.3, vmax=0.3)
ax.plot([1500, 17000], [1500, 17000], lw=3, color=colors.midnightblue)
ax.set_xlabel('Activity on weekdays [bpm]')
ax.set_ylabel('Activity on weekends [bpm]')

ax = axarr[1, 0]
plot2d_histograms(ax, df, 'v52', 'v53', bins=get_bins(-4, 10, 1/3), hline=7, vline=0)
ax.plot([-3, 1 + 1/3], [4, 8 + 1/3], lw=3, color=colors.midnightblue)
ax.set_xlabel('Sleep Onset [hrs relative to midnight]')
ax.set_ylabel('Sleep Offset [hrs relative to midnight]')

ax = axarr[1, 1]
bins = [get_bins(-5, 5, 1/3), get_bins(-5, 5, 1/3)]
plot2d_histograms(ax, df, 'v52weekday', 'v52weekend', bins=bins, vmin=-0.3, vmax=0.1)
ax.plot([-4 + 1/3, 1 + 2/3], [-4 + 1/3, 1 + 2/3], lw=3, color=colors.midnightblue)
ax.set_xlabel('Sleep Onset on weekdays\n[hrs relative to midnight]')
ax.set_ylabel('Sleep Onset on weekends\n[hrs relative to midnight]')

ax = axarr[1, 2]
bins = [get_bins(-5, 12, 1/3), get_bins(-5, 12, 1/3)]
plot2d_histograms(ax, df, 'v53weekday', 'v53weekend', bins=bins, vmin=-0.3, vmax=0.1)
ax.plot([5 - 2/3, 9], [5-2/3, 9], lw=3, color=colors.midnightblue)
ax.set_xlabel('Sleep Offset on weekdays\n[hrs relative to midnight]')
ax.set_ylabel('Sleep Offset on weekends\n[hrs relative to midnight]')

pos=6 * ['upper left']
pos[0] = 'upper right'
add_label(axarr.flatten(), pos=pos, size=22)

plt.tight_layout()

plt.savefig(OUTPUT_FOLDER / 'figure_X_2d_plots.pdf')

## Alternative version of Figure 3 with weekend/weekday discrimination

In [ ]:
f, axarr = plt.subplots(2, 2, figsize=(6, 6), sharey=True)

vital_key = 'v52'
bins = np.arange(0, 3.1, 1/3)
plot_survey_response_per_vitals(axarr[0, 0], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[0, 0], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')
plot_survey_response_per_vitals(axarr[0, 0], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

vital_key = 'v53'
bins = np.arange(0, 3.1, 1/3)
plot_survey_response_per_vitals(axarr[0, 1], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[0, 1], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')
plot_survey_response_per_vitals(axarr[0, 1], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

vital_key = 'midsleep'
bins = np.arange(0, 2.5, 1/3)
plot_survey_response_per_vitals(axarr[1, 0], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[1, 0], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')
plot_survey_response_per_vitals(axarr[1, 0], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

vital_key = 'v43'
bins = np.arange(20, 120, 10)
plot_survey_response_per_vitals(axarr[1, 1], df, f'{vital_key}stdweekend', 'total_wellbeing_Z', bins=bins, color=colors.pomegranate, err_fac=1.96, label='Weekends')
plot_survey_response_per_vitals(axarr[1, 1], df, f'{vital_key}stdweekday', 'total_wellbeing_Z', bins=bins, color=colors.greensea, err_fac=1.96, label='Weekdays')
plot_survey_response_per_vitals(axarr[1, 1], df, f'{vital_key}std', 'total_wellbeing_Z', bins=bins, color=colors.midnightblue, err_fac=1.96, label='All Days')

axarr[0, 0].set_xlabel('Standard Deviation in\nSleep Onset [hrs]')
axarr[0, 1].set_xlabel('Standard Deviation in\nSleep Offset [hrs]')
axarr[1, 0].set_xlabel('Standard Deviation in\nMidsleep [hrs]')
axarr[1, 1].set_xlabel('Standard Deviation in\nSleep Duration [mins]')

plt.tight_layout()

axarr[0, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')
axarr[1, 0].set_ylabel('Average WHO-5 Wellbeing Z-Scores')

axarr[0, 1].set_ylabel(None)
axarr[1, 1].set_ylabel(None)

add_label(axarr.flatten(), pos=['lower left', 'upper right', 'lower left', 'lower left'], size=22)

axarr[0, 1].legend(loc='lower left')

plt.savefig(OUTPUT_FOLDER / 'figure3_wellbeing_sleep_variation.pdf')

## Figure 4 - Weekend / weekday difference

In [ ]:
Zf, axarr = plt.subplots(1, 1, sharey=True, figsize=(6, 4))

question_key = 'total_wellbeing_Z'

bins = get_bins(-1, 3, .5)
plot_with_ttest(ax=axarr[0,0], vital_key ='social_jetlag', question_key=question_key, df=df, bins=bins)

bins = get_bins(-1.5, 3.5, .5)
plot_with_ttest(ax=axarr[0,1], vital_key ='v52difference', question_key=question_key, df=df, bins=bins)

bins = get_bins(-2, 4.5, 0.5)
plot_with_ttest(ax=axarr[0,2], vital_key ='v53difference', question_key=question_key, df=df, bins=bins)

bins = get_bins(-150, 180, 30)
plot_with_ttest(ax=axarr[1,0], vital_key ='v43difference', question_key=question_key, df=df, bins=bins)

bins = get_bins(-8000, 8001, 2000)
plot_with_ttest(ax=axarr[1,1], vital_key ='v9difference', question_key=question_key, df=df, bins=bins)

bins = get_bins(-8, 8, 2)
plot_with_ttest(ax=axarr[1,2], vital_key ='v65difference', question_key=question_key, df=df, bins=bins)

for ax in axarr[:, 1:].flatten():
    ax.set_ylabel(None)
    
for ax in axarr.flatten():
    ax.axhline(0, ls=':', color=colors.midnightblue)
    
for ax in axarr[:, 0].flatten():
    ax.set_ylabel('Average WHO-5 Z-Scores')

axarr[0, 0].set_xlabel('Social Jetlag [hrs]')

axarr[0, 1].set_xlabel('Sleep Onset Difference [hrs]')
axarr[0, 1].set_xticks([-1, 0, 1, 2, 3])

axarr[0, 2].set_xlabel('Sleep Offset Difference [hrs]')
axarr[0, 2].set_xticks([-1, 0, 1, 2, 3, 4])

axarr[1, 0].set_xlabel('Sleep Duration Difference [minutes]')
axarr[1, 0].set_xticks([-60, 0, 60, 120])

axarr[1, 1].set_xlabel('Activity Difference [steps]')
axarr[1, 1].set_xticks([-4000, 0, 4000])

axarr[1, 2].set_xlabel('RHR Difference [bpm]')
axarr[1, 2].set_xticks([-8, -6, -4, -2, 0, 2, 4, 6])

pos=6 * ['upper left']
add_label(axarr.flatten(), pos=pos, size=22)

plt.tight_layout()
plt.savefig(OUTPUT_FOLDER / 'figure4_wellbeing_weekend_weekday.pdf')